## Sleep-like slow oscillations improve visual classification through synaptic homeostasis and memory association in a thalamo-cortical model

### Authors: Bruna, Catalina Saini, Flavio Rusch

Project for:
Latin American Computational Neuroscience Summer School (LASCON) 2024, 
Universidade de Sao Paulo (USP), 
Sao Paulo, Brazil

Original paper: Capone, C., Pastorelli, E., Golosio, B., & Paolucci, P. S. (2019). Sleep-like slow oscillations improve visual classification through synaptic homeostasis and memory association in a thalamo-cortical model. Scientific Reports, 9(1), 1-11. https://doi.org/10.1038/s41598-019-45525-0

Simulation Engine: We performed spiking simulation using the NEST simulation engine, the high-performance general purpose simulator developed by the NEST Initiative, release 2.1231.

## Imports

In [1]:
import nest
import matplotlib as plt
import numpy as np


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.6.0
 Built: Oct 15 2023 15:12:16

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



In [2]:
dir(nest)

nest.ResetKernel()

## Network arquitecture: Minimal thalamo-cortical model

Upload image

 ## Neurons
 
 ### Adaptive exponential (AdEx) point like single compartment neurons

Insert image here.

In [3]:
cxn = nest.Create('aeif_cond_alpha', 780) # cortical excitation (Ncxn_1 = 180, Ncxn_2 = 600)
inn = nest.Create('aeif_cond_alpha', 200) # cortical inhibition
tcn = nest.Create('aeif_cond_alpha', 324) # thalamic excitation
ren = nest.Create('aeif_cond_alpha', 200) # thalamic inhibition

pg = nest.Create('poisson_generator') # contextual signal: poisson spike train generator

Notes: 

About the contextual signal: See https://nest-simulator.readthedocs.io/en/stable/models/poisson_generator.html.

## Neural connections

In [4]:
ep = 1.0 # connection probability

conn_spec_dict = {
    'rule': 
    'pairwise_bernoulli', # ?
    'p': ep
} 

nest.Connect(inn, cxn, conn_spec_dict, syn_spec={'synapse_model': 'stdp_synapse', 'weight': -4.0}) # in → cx
nest.Connect(cxn, inn, conn_spec_dict, syn_spec={'synapse_model': 'stdp_synapse', 'weight': 60.0}) # cx → in
nest.Connect(tcn, ren, conn_spec_dict, syn_spec={'synapse_model': 'stdp_synapse', 'weight': 10.0}) # tc → re
nest.Connect(ren, tcn, conn_spec_dict, syn_spec={'synapse_model': 'stdp_synapse', 'weight': -10.0}) # re → tc
nest.Connect(inn, inn, conn_spec_dict, syn_spec={'synapse_model': 'stdp_synapse', 'weight': -1.0}) # in → in
nest.Connect(ren, ren, conn_spec_dict, syn_spec={'synapse_model': 'stdp_synapse', 'weight': -1.0}) # re → re

Notes:
    
About connection management: See https://nest-simulator.readthedocs.io/en/v3.1/guides/connection_management.html

Questions:
Pairwise Bernoulli?
STDP synapse?

### STDP plasticity

Insert image here.

In [5]:
# plastic neurons
wmin = 1.0 # minimum synaptic weight
a = 1.0 # symmetric STPD

nest.Connect(cxn, cxn, conn_spec_dict, {'synapse_model': 'stdp_synapse', 'alpha': a, 'Wmax': 15.0}) # cx → cx
nest.Connect(cxn, tcn, conn_spec_dict, {'synapse_model': 'stdp_synapse', 'alpha': a, 'Wmax': 13.0}) # cx → tc
nest.Connect(tcn, cxn, conn_spec_dict, {'synapse_model': 'stdp_synapse', 'alpha': a, 'Wmax': 5.5}) # tc → cx

Notes:
    
STDP synapses: https://nest-simulator.readthedocs.io/en/v2.20.0/models/stdp.html
Synapses: https://nest-simulator.readthedocs.io/en/v2.18.0/models/synapses.html

Questions:
How to include wmin?

In [6]:
print(nest.num_connections)

1635440


## Image preprocessing

In [7]:
def image_preprocessing():
    None

## Simulation

In [ ]:
class Simulation(): # Network structure and dynamics are compared in the pre- and post- sleep phases.

    def training(): # Training phase
        ''' Visual patterns are learned by the network
    
        Training set (TS) runs:
            1. 9 imgs, 3 dif ex for 3 dif digits (0, 1, 2), Ncx = 180. 
            2. 30 imgs, 3 ex per digit, Ncx = 600. 
        
        '''
    
        # Groups of 20 cortical neurons per image.
        None
        
        # Device connections
        nest.Connect(pg, cxn, syn_spec={'weight': 8.0}) # pg inputs
        nest.Connect(pg, inn, syn_spec={'weight': 15.0})

        # 10 kHz pg  provided to inn (prevents trained neurons to respond to new stimuli.
        pg.set(rate=10000.0)
        
        # For every img through the thalamic pathway -> 2 kHz pg to a dif set of 20 neurons.
        pg.set(rate=2000.0)
        
    def slow_oscillations(): # Post-training phase (pre-sleep phase?)
        '''Sleep-like thalamo-cortical spontaneous slow oscillations activity is induced '''
        
        # For 600s by providing a non-specific Poisson noise inside the cortex (700 Hz).
        pg.set(rate=700.0)
        
        # and increasing the strength of SFA parameter (b = 60, in eq. (1)). 
        None
        
        # No external stimulus is provided to tc cells. 
        None
        
        # The synaptic weights between inn and exn is reduced to w in→cx = −0.5. 
        None
        
        # Asymmetric STDP plasticity (α = 3.0) in the recurrent cx connectivity. 
        None
   
    def retrieval(): # Pre-sleep retrieval phase
        ''' Patterns are recalled '''
        
        nest.Connect(pg, tcn, syn_spec={'weight': 5.0})
        pg.set(rate=80000.0)
    
    def sleep(): # Sleep phase
        None
        
    def exercise(): # Post-sleep phase (option 1)
        ''' Network is exercised on the retrieval of previous learned patterns'''
        
        None
    
    def classification(): # Post-sleep phase (option 2)
        ''' Applied to the classification of novel examples'''
        
        None
    
    nest.Simulate(t=2000)

Questions: 
1. conn_spec_dict for pg? 
2. syn_spec model for pg?
3. pg, tcn and tcn, pg? 
4. rate is in Hz or kHz? 
5. slow oscillations phase is the same as pre-sleep phase?